In [1]:
import nltk
from konlpy.corpus import kolaw
from konlpy.tag import *
import pandas as pd
import platform
from collections import Counter
from matplotlib import font_manager, rc
from konlpy.tag import Twitter; t = Twitter()
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
dfa = pd.read_csv("../petition_data_all.csv")

In [3]:
dfa = dfa[dfa["count"]>=1000]

In [4]:
dfa

,num,category,start-days,end-days,person,progress,title,count,petition_overview
7,28,경제민주화,2017-08-19,2017-08-26,naver - ***,청원종료,소액주주를 보호해주십시오,2137,\n\t\t\t\t\t\t\t\t\t\t\t** 존경하옵는 문재인대통령님께 \n...
21,43,육아/교육,2017-08-19,2017-09-18,naver - ***,청원종료,기간제 교사의 정규직화를 반대합니다.,11293,"\n\t\t\t\t\t\t\t\t\t\t\t대통령님, 안녕하세요. 저는 임용을 준비..."
24,46,육아/교육,2017-08-19,2017-09-18,naver - ***,청원종료,기간제교사의 정규직화를 반대합니다.,1933,"\n\t\t\t\t\t\t\t\t\t\t\t대통령님, 안녕하세요. 저는 임용을 준비..."
28,50,일자리,2017-08-19,2017-10-18,naver - ***,청원종료,치과위생사 국가고시 날짜 변경 억울합니다,1251,\n\t\t\t\t\t\t\t\t\t\t\t안녕하십니까? 대통령님 간단하게 제소개 ...
38,61,일자리,2017-08-19,2017-08-26,naver - ***,청원종료,영어회화전문강사제도는 유지되어야합니다.,1185,\n\t\t\t\t\t\t\t\t\t\t\t1. 영전강의고용안정을 원합니다 \n\t...
46,69,육아/교육,2017-08-20,2017-09-04,kakao - ***,청원종료,"영어전문강사, 스포츠전문강사를 무기계약직 혹은 정규직으로 전환하는 것에 반대합니다",4934,\n\t\t\t\t\t\t\t\t\t\t\t저는 30대 초반에 수능 다시 공부해서 ...
48,71,육아/교육,2017-08-20,2017-09-19,naver - ***,청원종료,"영양 교사 폐지, 영양직 직렬 신설을 청원합니다.",4484,\n\t\t\t\t\t\t\t\t\t\t\t대통령님께서 대선 후보이시던 시절 충격적...
52,75,육아/교육,2017-08-20,2017-09-04,naver - ***,청원종료,수능 상대평가 유지 정시확대,6420,\n\t\t\t\t\t\t\t\t\t\t\t기회의 사다리이자 가장 공정한 수능 상대...
66,89,일자리,2017-08-20,2017-11-18,naver - ***,청원종료,고용노동부 일반상담원 처우개선을 위한 예산편성 촉구 청원서,1101,\n\t\t\t\t\t\t\t\t\t\t\t문재인정부 출범 100일을 맞이하는 기자...
81,105,육아/교육,2017-08-20,2017-09-19,naver - ***,청원종료,중등교과교사티오 증원을 요청합니다.(제발 읽어주세요),2212,\n\t\t\t\t\t\t\t\t\t\t\t저는 중등학교 현직교사입니다. \n\t...


In [5]:
wdf = pd.read_csv("week_dict.csv")
week_dict = {}
for row in wdf.itertuples(index=True, name='Pandas'):
    week_dict[getattr(row, "week")] = getattr(row, "days").split(',')
week_dict

{'2017-08-04': ['2017-08-19', '2017-08-20'],
 '2017-08-05': ['2017-08-21',
  '2017-08-22',
  '2017-08-23',
  '2017-08-24',
  '2017-08-25',
  '2017-08-26',
  '2017-08-27'],
 '2017-09-01': ['2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31'],
 '2017-09-02': ['2017-09-01', '2017-09-02', '2017-09-03'],
 '2017-09-03': ['2017-09-04',
  '2017-09-05',
  '2017-09-06',
  '2017-09-07',
  '2017-09-08',
  '2017-09-09',
  '2017-09-10'],
 '2017-09-04': ['2017-09-11',
  '2017-09-12',
  '2017-09-13',
  '2017-09-14',
  '2017-09-15',
  '2017-09-16',
  '2017-09-17'],
 '2017-10-02': ['2017-09-18',
  '2017-09-19',
  '2017-09-20',
  '2017-09-21',
  '2017-09-22',
  '2017-09-23',
  '2017-09-24'],
 '2017-10-03': ['2017-09-25',
  '2017-09-26',
  '2017-09-27',
  '2017-09-28',
  '2017-09-29',
  '2017-09-30'],
 '2017-10-04': ['2017-10-01'],
 '2017-11-01': ['2017-10-02',
  '2017-10-03',
  '2017-10-04',
  '2017-10-05',
  '2017-10-06',
  '2017-10-07',
  '2017-10-08'],
 '2017-11-02': ['2017-10-09',
  '2017-10-10',


In [9]:
week_keyword = []
for week_n in week_dict:
    test_index = dfa[dfa["num"]==0]
    for wd in week_dict[week_n]:
        df1 = test_index.append(dfa[dfa["start-days"] == wd])
    text = list(df1["petition_overview"])
    text = ' '.join(text)
    ko_con_text = text
    print(len(text))
    tokens_ko = t.nouns(ko_con_text)
    stop_words = ['직','제','저', '이','그','때','및','것이','합니다.','\\n',':','','거','수','그' ,"합","이","있는","있습니다.","것","및","하지만","때문에"]
    tokens_ko = [each_word for each_word in tokens_ko if each_word not in stop_words]
    text = ' '.join(tokens_ko)
    ko = nltk.Text(tokens_ko, name= '청원')
    data = ko.vocab().most_common(500)
    tmp_data = dict(data)
    week_keyword.append(tmp_data)

20719
488
9889
11084
1309
476
0
4
3836
1027
6030
905
955
1234
14796
225
5121
7694
18722
11673
7231
3079
509
232
2533
8563
8462
6781
10401
5014
4275
6829
9899
3802
2321
11708
3168
12340
3607
13672
2475
1254
6825
7073
28003
4635
3706
9247
7748
11638
7774
17434
4768
3911
21493
4486
8591
12131
26409
13460
2804
6058
2398
7252
1525
7241
16368
14585
8486


In [11]:
week_keyword[0]

{'교사': 73,
 '수업': 51,
 '강사': 48,
 '교육': 37,
 '문제': 30,
 '영어': 29,
 '명': 28,
 '학교': 27,
 '상담': 27,
 '업무': 26,
 '교과': 25,
 '학생': 25,
 '정책': 24,
 '정부': 24,
 '등': 23,
 '심리': 23,
 '과목': 22,
 '체육': 22,
 '자격': 19,
 '어회': 17,
 '전문성': 17,
 '생각': 17,
 '제도': 17,
 '우리': 17,
 '관련': 16,
 '계약': 16,
 '스포츠': 16,
 '적용': 16,
 '담임': 15,
 '더': 15,
 '피해': 15,
 '주택': 15,
 '현장': 14,
 '대한': 14,
 '치료': 14,
 '심리치료': 14,
 '소급': 14,
 '기간': 13,
 '초등': 13,
 '기관': 12,
 '심리치료사': 12,
 '규정': 12,
 '때문': 11,
 '무기계약직': 11,
 '교육과정': 11,
 '전문가': 11,
 '노동부': 11,
 '센터': 11,
 '못': 11,
 '발생': 11,
 '외상': 11,
 '임': 11,
 '분양': 11,
 '교대': 10,
 '자격증': 10,
 '요구': 10,
 '일': 10,
 '이상': 10,
 '고용노동부': 10,
 '투쟁': 10,
 '책임': 10,
 '후': 10,
 '정교': 10,
 '권한': 10,
 '상황': 10,
 '요건': 10,
 '피해자': 10,
 '정': 9,
 '과정': 9,
 '사회': 9,
 '내': 9,
 '모두': 9,
 '차': 9,
 '정규': 9,
 '성폭력': 9,
 '한국': 9,
 '대출': 9,
 '무': 9,
 '사람': 8,
 '중': 8,
 '선생님': 8,
 '증원': 8,
 '질': 8,
 '학': 8,
 '하나': 8,
 '행정': 8,
 '이명박': 8,
 '단독': 8,
 '융합': 8,
 '위해': 8,
 '발급': 8,
 '금': 8,
 '수능':

In [12]:
rdf = pd.read_csv("rating_information.csv")
rdf

,차이,추천 1000개 이상,추천 10000개 이상,추천 200000개 이상,청원전체,지지율,대통령 주
0,0.00,14,1,0,127,76.45,14
1,-1.90,21,1,0,537,74.55,15
2,-4.00,13,2,0,507,70.55,16
3,-2.50,12,4,1,669,68.05,17
4,-0.25,29,6,1,11344,67.80,18
5,-1.45,10,3,0,977,66.35,19
6,4.40,18,0,0,2334,70.75,21
7,-1.85,9,3,1,1716,68.90,22
8,1.20,2,2,0,135,70.10,23
9,1.55,9,3,0,1312,71.65,24


In [15]:
list(rdf["대통령 주"])

[14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84]

In [58]:
clist = list(rdf["대통령 주"])
clist.append("keyword")
test = pd.DataFrame(columns=clist)
test

,14,15,16,17,18,19,21,22,23,24,...,76,77,78,79,80,81,82,83,84,keyword


In [33]:
all_set = []
for kw in week_keyword:
    all_set.extend(list(kw.keys()))
all_set = list(set(tokens_ko))

In [34]:
all_set

['민중',
 '강도',
 '키네',
 '사기',
 '호황',
 '연신내',
 '십',
 '위장',
 '서점',
 '마나',
 '국무회의',
 '제풀이',
 '인스타',
 '이의제기',
 '초미세먼지',
 '문제점',
 '나영이',
 '무력',
 '지진',
 '바이러스',
 '성범죄자',
 '법도',
 '성신여대',
 '잔',
 '교장',
 '훈장',
 '견',
 '고가',
 '더',
 '입항',
 '허재용',
 '범입',
 '박정희',
 '밖',
 '디테일',
 '무려',
 '바치',
 '확율',
 '실수',
 '주택',
 '바이퍼',
 '계층',
 '밥',
 '노년',
 '자동',
 '명',
 '교직원',
 '아래층',
 '확률',
 '자임',
 '김수현',
 '의무',
 '논문',
 '전통',
 '팀',
 '임상시험',
 '호출',
 '유입',
 '쉐보레',
 '여자친구',
 '고정연',
 '노컷뉴스',
 '보육',
 '알뜰살뜰',
 '전횡',
 '매일',
 '국군',
 '일어난다',
 '증명',
 '트위터',
 '매장',
 '복원',
 '가게',
 '남대서양',
 '림프',
 '창원',
 '사태',
 '사탕',
 '특권',
 '퇴근',
 '순간',
 '형성',
 '외국인',
 '직영',
 '지명',
 '일산',
 '주사',
 '랍시고',
 '세습',
 '항변',
 '구조조정',
 '노력',
 '오크',
 '신선',
 '물가',
 '전전',
 '유라',
 '이바지',
 '제작',
 '문구',
 '주차',
 '공백',
 '말로',
 '피',
 '변형',
 '학생',
 '로서',
 '공모전',
 '더욱이',
 '편견',
 '미사',
 '옥트레오티드',
 '첫걸음',
 '등',
 '나로',
 '젊은이',
 '극히',
 '쥐',
 '허브',
 '주정',
 '여객기',
 '무엇',
 '국정',
 '간호',
 '측은',
 '번영',
 '동물',
 '차기',
 '과천',
 '역자',
 '돌이',
 '그림',
 '이기',
 '불구속',
 '배정',
 '중차',
 '방법

In [35]:
nlist = list(rdf["대통령 주"])

In [36]:
nlist

[14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84]

In [37]:
len(all_set)

7259

In [44]:
conut_lists = []
for wk in week_keyword:
    conut_list = []
    for one_set in all_set:
        try:
            conut_list.append(wk[one_set])
        except:
            conut_list.append(0)
    conut_lists.append(conut_list)

In [51]:
len(conut_lists),len(conut_lists[0])

(69, 7259)

In [50]:
len(all_set),len(conut_lists)

(7259, 69)

In [59]:
for i in range(len(all_set)):
    tlist = []
    for j in range(len(conut_lists)):
        tlist.append(conut_lists[j][i])
    tlist.append(all_set[i])
    test.loc[i] = tlist
    
test

,14,15,16,17,18,19,21,22,23,24,...,76,77,78,79,80,81,82,83,84,keyword
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,민중
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,강도
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,키네
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,사기
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,호황
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,연신내
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,십
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,위장
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,서점
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,마나


In [61]:
test[test["keyword"]=="교사"]

,14,15,16,17,18,19,21,22,23,24,...,76,77,78,79,80,81,82,83,84,keyword
6655,73,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,18,0,9,교사


In [62]:
82 - 14

68

In [69]:
week_keyword[3]["교사"]

2

In [54]:
len(list(test))

69

In [39]:
len(week_keyword)

69

In [73]:
rdf[rdf["차이"]<0]

,차이,추천 1000개 이상,추천 10000개 이상,추천 200000개 이상,청원전체,지지율,대통령 주
1,-1.90,21,1,0,537,74.55,15
2,-4.00,13,2,0,507,70.55,16
3,-2.50,12,4,1,669,68.05,17
4,-0.25,29,6,1,11344,67.80,18
5,-1.45,10,3,0,977,66.35,19
7,-1.85,9,3,1,1716,68.90,22
14,-0.85,17,6,1,1160,72.40,29
15,-3.10,21,6,0,15937,69.30,30
17,-1.40,48,9,1,5399,68.50,32
20,-5.30,39,7,0,4483,66.50,35


In [75]:
test["alls"]= test.sum(axis = 1, skipna = True)
test

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,민중,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,강도,12
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,키네,2
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,사기,14
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,호황,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,연신내,2
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,십,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,위장,2
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,서점,14
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,마나,3


In [77]:
test.sort_values(by=['alls'],ascending = False)

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
1050,7,2,6,7,0,0,0,0,1,0,...,4,1,0,8,9,8,14,0,국민,424
4744,6,0,2,8,0,0,0,0,5,0,...,0,0,2,4,4,52,0,7,아이,419
113,23,0,4,17,0,1,0,0,0,1,...,3,1,12,0,11,13,11,10,등,391
6420,8,0,2,9,0,1,0,0,1,0,...,4,5,0,1,4,5,5,4,사람,349
1185,17,1,8,9,1,2,0,0,3,0,...,1,1,0,0,7,16,20,9,우리,333
6651,17,0,1,9,2,0,0,0,6,0,...,3,7,2,0,2,10,10,2,생각,292
1985,24,0,4,1,5,0,0,0,1,1,...,2,4,2,1,3,3,15,1,정부,288
6655,73,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,18,0,9,교사,285
626,30,0,4,0,0,0,0,0,2,0,...,2,0,5,0,7,13,5,4,문제,281
2503,3,0,3,2,0,0,0,0,4,0,...,1,2,6,2,4,7,13,5,말,270


In [82]:
test.to_csv("week_keyword_up1000.csv",index=False)

In [89]:
test = pd.read_csv("week_keyword_up1000.csv")
test

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,민중,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,강도,12
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,키네,2
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,사기,14
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,호황,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,연신내,2
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,십,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,위장,2
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,서점,14
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,마나,3


In [86]:
test_b = test.astype('bool')
test_b = test_b.astype(float)
test_b["keyword"] = test["keyword"]
test_b["alls"]= test_b.sum(axis = 1, skipna = True)
test_b

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,민중,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,강도,7.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,키네,2.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,사기,9.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,호황,2.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,연신내,2.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,십,2.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,위장,3.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서점,2.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,마나,3.0


In [90]:
test_b = test_b.sort_values(by=['alls'],ascending = False)

In [91]:
test_b.to_csv("week_keyword_up1000_bool.csv",index=False)

In [92]:
test_b = pd.read_csv("week_keyword_up1000_bool.csv")
test_b

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,사람,58.0
1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,등,56.0
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,국민,56.0
3,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,정부,56.0
4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,일,55.0
5,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,말,54.0
6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,생각,54.0
7,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,더,53.0
8,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,상황,52.0
9,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,대한,50.0


In [93]:
test = test.sort_values(by=['alls'],ascending = False)
test

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
1050,7,2,6,7,0,0,0,0,1,0,...,4,1,0,8,9,8,14,0,국민,424
4744,6,0,2,8,0,0,0,0,5,0,...,0,0,2,4,4,52,0,7,아이,419
113,23,0,4,17,0,1,0,0,0,1,...,3,1,12,0,11,13,11,10,등,391
6420,8,0,2,9,0,1,0,0,1,0,...,4,5,0,1,4,5,5,4,사람,349
1185,17,1,8,9,1,2,0,0,3,0,...,1,1,0,0,7,16,20,9,우리,333
6651,17,0,1,9,2,0,0,0,6,0,...,3,7,2,0,2,10,10,2,생각,292
1985,24,0,4,1,5,0,0,0,1,1,...,2,4,2,1,3,3,15,1,정부,288
6655,73,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,18,0,9,교사,285
626,30,0,4,0,0,0,0,0,2,0,...,2,0,5,0,7,13,5,4,문제,281
2503,3,0,3,2,0,0,0,0,4,0,...,1,2,6,2,4,7,13,5,말,270


In [94]:
test.to_csv("week_keyword_up1000.csv",index=False)

In [95]:
test = pd.read_csv("week_keyword_up1000.csv")
test

,14,15,16,17,18,19,21,22,23,24,...,77,78,79,80,81,82,83,84,keyword,alls
0,7,2,6,7,0,0,0,0,1,0,...,4,1,0,8,9,8,14,0,국민,424
1,6,0,2,8,0,0,0,0,5,0,...,0,0,2,4,4,52,0,7,아이,419
2,23,0,4,17,0,1,0,0,0,1,...,3,1,12,0,11,13,11,10,등,391
3,8,0,2,9,0,1,0,0,1,0,...,4,5,0,1,4,5,5,4,사람,349
4,17,1,8,9,1,2,0,0,3,0,...,1,1,0,0,7,16,20,9,우리,333
5,17,0,1,9,2,0,0,0,6,0,...,3,7,2,0,2,10,10,2,생각,292
6,24,0,4,1,5,0,0,0,1,1,...,2,4,2,1,3,3,15,1,정부,288
7,73,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,18,0,9,교사,285
8,30,0,4,0,0,0,0,0,2,0,...,2,0,5,0,7,13,5,4,문제,281
9,3,0,3,2,0,0,0,0,4,0,...,1,2,6,2,4,7,13,5,말,270
